<a href="https://colab.research.google.com/github/abdo96/Spoiled-Food-Detectors/blob/master/Spoiled_Product_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
Created on Jul 11, 2017
Modified to Keras on OCT 2018
'''""
from keras.utils import plot_model
#from IPython.display import Image
from datetime import datetime
import logging
import tensorflow as tf
import math
from keras import layers
from sklearn.cluster import KMeans
import time
import keras
from keras import regularizers
import signal
from keras.engine import sequential
import keras.backend as K
import re
import numpy as np
from keras.layers import BatchNormalization,Input,MaxPooling2D,AveragePooling2D,Conv2D,Dropout,Flatten,Dense,Activation
import numpy
import json
from copy import deepcopy
import argparse



Using TensorFlow backend.


In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1IdWQwGA-imIUFTxsMeFvtQJFXBdh0Mxb' in parents"}).GetList()

for f in file_list:
 uploaded = drive.CreateFile({f['title']: f['id']})
 uploaded.SetContentString('Sample upload file content')
 uploaded.Upload()

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%ls /gdrive/'My Drive'/'Colab Notebooks'/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
 CancerPro.ipynb                            self.predictEncoding.png
'Copy of Spoiled_Product_Detection.ipynb'   self.predictReconstruction.png
 data/                                      self.trainAutoencoder.png
 Rotten_apple_and_bananas/                  Spoiled_Product_Detection.ipynb


In [0]:
logFormatter = logging.Formatter("[%(asctime)s]  %(message)s", datefmt='%m/%d %I:%M:%S')

rootLogger = logging.getLogger()
rootLogger.setLevel(logging.DEBUG)

fileHandler = logging.FileHandler(datetime.now().strftime('/gdrive/My Drive/Colab Notebooks/Rotten_apple_and_bananas/logs/dcjc_%H_%M_%d_%m.log'))
fileHandler.setFormatter(logFormatter)
rootLogger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logFormatter)
rootLogger.addHandler(consoleHandler)

In [0]:
'''
Created on Jul 11, 2017
'''

import _pickle as cPickle
import _pickle 
import gzip
from skimage import transform 
import numpy as np
from PIL import Image
import matplotlib
import os
# For plotting graphs via ssh with no display
# Ref: https://stackoverflow.com/questions/2801882/generating-a-png-with-matplotlib-when-display-is-undefined
matplotlib.use('Agg')
from keras.preprocessing.image import load_img
from matplotlib import pyplot as plt
from numpy import float32
from sklearn import metrics
from sklearn.cluster.k_means_ import KMeans
from sklearn import manifold
from sklearn.utils.linear_assignment_ import linear_assignment
from sklearn import preprocessing
from sklearn.utils import shuffle

class DatasetHelper(object):
    '''
    Utility class for handling different datasets
    '''

    def __init__(self, name,typeOfData):
        '''
        A dataset instance keeps dataset name, the input set, the flat version of input set
        and the cluster labels
        '''
        self.name = name
        self.typeOfData = typeOfData
        if name == 'apples' or 'bananas' or 'oranges':
            self.dataset = ROTPROdataset()
       
            

    def loadDataset(self):
        '''
        Load the appropriate dataset based on the dataset name
        '''
        if self.name == 'apples':
             self.input, self.labels, self.input_flat = self.dataset.loadDataset(self.name,self.typeOfData)
        elif self.name == 'oranges':
            self.input, self.labels, self.input_flat = self.dataset.loadDataset(self.name,self.typeOfData)
        elif self.name == 'bananas':
            self.input, self.labels, self.input_flat = self.dataset.loadDataset(self.name,self.typeOfData)
        else:
            self.input, self.labels, self.input_flat = self.dataset.loadDataset()
            
    def getClusterCount(self):
        '''
        Number of clusters in the dataset - e.g 10 for mnist, 20 for coil20
        '''
        return self.dataset.cluster_count

      
    def get_shape(self):
        '''
         shape of datasets 
        '''    
        return self.dataset.shape
    def iterate_minibatches(self, set_type, batch_size, targets=None, shuffle=False):
        '''
        Utility method for getting batches out of a dataset
        :param set_type: IMAGE - suitable input for CNNs or FLAT - suitable for DNN
        :param batch_size: Size of minibatches
        :param targets: None if the output should be same as inputs (autoencoders), otherwise takes a target array from which batches can be extracted. Must have the same order as the dataset, e.g, dataset inputs nth sample has output at target's nth element
        :param shuffle: If the dataset needs to be shuffled or not
        :return: generates a batches of size batch_size from the dataset, each batch is the pair (input, output)
        '''
        inputs = None
        if set_type == 'IMAGE':
            inputs = self.input
            if targets is None:
                targets = self.input
        elif set_type == 'FLAT':
            inputs = self.input_flat
            if targets is None:
                targets = self.input_flat
        assert len(inputs) == len(targets)
        if shuffle:
            indices = np.arange(len(inputs))
            np.random.shuffle(indices)
        for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batch_size]
            else:
                excerpt = slice(start_idx, start_idx + batch_size)
            yield inputs[excerpt], targets[excerpt]


class MNISTDataset(object):
    '''
    Class for reading and preparing MNIST dataset
    '''

    def __init__(self):
        self.cluster_count = 10

    def loadDataset(self):
        f = gzip.open('mnist/mnist.pkl.gz', 'rb')
        train_set, _, test_set = cPickle.load(f,encoding='latin1')
        train_input, train_input_flat, train_labels = self.prepareDatasetForAutoencoder(train_set[0], train_set[1])
        test_input, test_input_flat, test_labels = self.prepareDatasetForAutoencoder(test_set[0], test_set[1])
        f.close()
        # combine test and train samples
        print(np.concatenate((train_labels, test_labels)).shape)
        return [np.concatenate((train_input, test_input)), np.concatenate((train_labels, test_labels)),
                np.concatenate((train_input_flat, test_input_flat))]

    def prepareDatasetForAutoencoder(self, inputs, targets):
        '''
        Returns the image, flat and labels as a tuple
        '''
        X = inputs
        X = X.reshape((-1,28, 28,1))
        return (X, X.reshape((-1, 28 * 28)), targets)


class STLDataset(object):
    '''
    Class for preparing and reading the STL dataset
    '''

    def __init__(self):
        self.cluster_count = 10

    def loadDataset(self):
        train_x = np.fromfile('stl/train_X.bin', dtype=np.uint8)
        train_y = np.fromfile('stl/train_y.bin', dtype=np.uint8)
        test_x = np.fromfile('stl/train_X.bin', dtype=np.uint8)
        test_y = np.fromfile('stl/train_y.bin', dtype=np.uint8)
        train_input = np.reshape(train_x, (-1, 3, 96, 96))
        train_labels = train_y
        train_input_flat = np.reshape(test_x, (-1, 1, 3 * 96 * 96))
        test_input = np.reshape(test_x, (-1, 3, 96, 96))
        test_labels = test_y
        test_input_flat = np.reshape(test_x, (-1, 1, 3 * 96 * 96))
        print(test_labels.shape)
        return [np.concatenate(train_input, test_input), np.concatenate(train_labels, test_labels),
                np.concatenate(train_input_flat, test_input_flat)]


 
class ROTPROdataset(object):
    '''
    Class for reading and preparing RottenPRO dataset
    '''
    def __init__(self):
        self.cluster_count = 2
        
    def loadDataset(self,typOfLabels = ' ',typOfData = ' '):
      
      data= []
      labels =[]
      labelsName = []
      if typOfLabels == 'apples':
           labelsName =['rottenapples','freshapples']
      elif typOfLabels =='oranges':
           labelsName =['rottenoranges','freshoranges']
      elif typOfLabels == 'bananas':
           labelsName =['rottenbanana','freshbanana']
          
      if typOfData == 'Train':
            root = '/gdrive/My Drive/Colab Notebooks/Rotten_apple_and_bananas/train/'+str(typOfLabels)+'/'
      elif typOfData == 'Test':    
            root ='/gdrive/My Drive/Colab Notebooks/Rotten_apple_and_bananas/test/'+str(typOfLabels)+'/'
      
      for rootName,dirName,fileNames in os.walk(root):
          if(not rootName == root):
             for fileName in fileNames:
                  img = load_img(rootName+'/'+fileName)
                  if rootName.split('/')[-1] == labelsName[0]:
                           labels+=[0]
                  elif rootName.split('/')[-1]== labelsName[1]:
                           #print('1111')
                           labels+=[1]
                
                  #imagArray= np.array(img)
                  transformed=transform.resize(np.array(img),(331,331))
                  #imgGray = np.array(imgGray)
                  #transformed.reshape((transformed.shape[0],transformed.shape[1],3))
                  data += [transformed.reshape((transformed.shape[0],transformed.shape[1],3))]
                  

      #print(data)        
      data=np.stack(data)
      labels=np.stack(labels)
      data,labels= shuffle(data,labels,random_state=0)
      
      data_flat = data.reshape((-1,data.shape[1]*data.shape[2]*3))
      #features_flat =data.reshape(-1,data.shape[0]*train_x.shape[1])
      return [data,labels,data_flat] 
      
def visualizeData(Z, labels, num_clusters, title):
    '''
    TSNE visualization of the points in latent space Z
    :param Z: Numpy array containing points in latent space in which clustering was performed
    :param labels: True labels - used for coloring points
    :param num_clusters: Total number of clusters
    :param title: filename where the plot should be saved
    :return: None - (side effect) saves clustering visualization plot in specified location
    '''
    labels = labels.astype(int)
    print(Z.shape)
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
    Z_tsne = tsne.fit_transform(Z)
    fig = plt.figure()
    plt.scatter(Z_tsne[:, 0], Z_tsne[:, 1], s=2, c=labels, cmap=plt.cm.get_cmap("jet", num_clusters))
    plt.colorbar(ticks=range(num_clusters))
    fig.savefig(title, dpi=fig.dpi)

[09/22 06:24:49]  Loaded backend agg version unknown.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
import keras.applications as kerasA

In [0]:
ResNet101=kerasA.resnet.ResNet101(input_shape=(331,331,3),weights=None,classes=1)

[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2041: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



[09/19 10:42:54]  From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [0]:
ROTPROData=DatasetHelper('oranges','Train')
ROTPROData.loadDataset()

[09/22 06:25:46]  STREAM b'IHDR' 16 13
[09/22 06:25:46]  STREAM b'IDAT' 41 8192
[09/22 06:25:46]  STREAM b'IHDR' 16 13
[09/22 06:25:46]  STREAM b'IDAT' 41 8192
[09/22 06:25:46]  STREAM b'IHDR' 16 13
[09/22 06:25:46]  STREAM b'IDAT' 41 8192
[09/22 06:25:46]  STREAM b'IHDR' 16 13
[09/22 06:25:46]  STREAM b'IDAT' 41 8192
[09/22 06:25:47]  STREAM b'IHDR' 16 13
[09/22 06:25:47]  STREAM b'IDAT' 41 8192
[09/22 06:25:47]  STREAM b'IHDR' 16 13
[09/22 06:25:47]  STREAM b'IDAT' 41 8192
[09/22 06:25:47]  STREAM b'IHDR' 16 13
[09/22 06:25:47]  STREAM b'IDAT' 41 8192
[09/22 06:25:48]  STREAM b'IHDR' 16 13
[09/22 06:25:48]  STREAM b'IDAT' 41 8192
[09/22 06:25:48]  STREAM b'IHDR' 16 13
[09/22 06:25:48]  STREAM b'iCCP' 41 3105
[09/22 06:25:48]  iCCP profile name b'ICC Profile'
[09/22 06:25:48]  Compression method 0
[09/22 06:25:48]  STREAM b'pHYs' 3158 9
[09/22 06:25:48]  STREAM b'iTXt' 3179 413
[09/22 06:25:48]  STREAM b'iDOT' 3604 28
[09/22 06:25:48]  b'iDOT' 3604 28 (unknown)
[09/22 06:25:48]  STREA

In [0]:

ResNet101.compile(optimizer='Adam',loss=['binary_crossentropy'],metrics=['binary_accuracy','categorical_accuracy'])
ResNet101.fit(ROTPROData.input,labelsReshaped,validation_split=0.3,epochs=20)

[09/19 11:00:54]  From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[09/19 11:00:54]  From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2830 samples, validate on 1213 samples
Epoch 1/20
2830/2830 [==============================] - 6623s 2s/step - loss: 9.2105 - binary_accuracy: 0.4223 - categorical_accuracy: 1.0000 - val_loss: 9.3972 - val_binary_accuracy: 0.4106 - val_categorical_accuracy: 1.0000
Epoch 2/20
 256/2830 [=>............................] - ETA: 1:30:39 - loss: 9.4035 - binary_accuracy: 0.4102 - categorical_accuracy: 1.0000

KeyboardInterrupt: ignored

In [0]:
labelsReshaped=ROTPROData.labels.reshape(ROTPROData.labels.shape[0],1)

In [0]:
ROTPROData.labels.shape

In [0]:
def create_model(input_shape=(512,512,3),filter_size=(3,3),nOfunits=[32],nOfLayers =1,classes=1,acticvation_inter='relu',activation_out='sigmoid',optimizer='rmsprop',loss=['binary_crossentropy'],metrics=['accuracy'],pool_size):
    '''
      function takes the five parameters and create a CNN model for classification and returns it.
    '''
    input_shape = input_shape
    filter_size = filter_size
    num_OF_units = nOfunits
    nOfLayers = nOfLayers
    model = sequential.Sequential()
    model.add(Conv2D(num_OF_units[0],filter_size,input_shape=input_shape))
    model.add(Activation(activation_inter))
    model.add(MaxPooling2D(pool_size=pool_size))
    model = sequential.Sequential()
    for indxLyer in range(1,nOfLayers):
        model.add(Conv2D(num_OF_units[indxLyer],filter_size))
        model.add(Activation(activation_inter))
        model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(classes,activation=activation_out))
    model.compile(optimizer=optimizer,loss=loss,metrics=metrics)


SyntaxError: ignored

In [0]:

model.summary()

ValueError: ignored

In [0]:
model.fit(ROTPROData.input,labelsReshaped,validation_split=0.3,epochs=30)

Train on 2142 samples, validate on 919 samples
Epoch 1/30
2142/2142 [==============================] - 32s 15ms/step - loss: 0.5625 - acc: 0.7166 - val_loss: 0.3590 - val_acc: 0.8564
Epoch 2/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.3930 - acc: 0.8352 - val_loss: 0.3617 - val_acc: 0.8433
Epoch 3/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.3055 - acc: 0.8973 - val_loss: 0.2517 - val_acc: 0.9162
Epoch 4/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.2823 - acc: 0.8992 - val_loss: 0.2524 - val_acc: 0.9086
Epoch 5/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.2286 - acc: 0.9216 - val_loss: 0.2145 - val_acc: 0.9195
Epoch 6/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.2161 - acc: 0.9248 - val_loss: 0.1804 - val_acc: 0.9271
Epoch 7/30
2142/2142 [==============================] - 28s 13ms/step - loss: 0.1950 - acc: 0.9384 - val_loss: 0.1986 - val_acc: 0.9249
E

In [0]:
model.save_weights('/gdrive/My Drive/Colab Notebooks/Rotten_apple_and_bananas/saved_params/model_'+str(ROTPROData.name)+'30epochs_128unit_last_acc_0.9897_val_acc_0.9728_test_acc_0.9709.h5')

In [0]:
model.save('')

4043/4043 [==============================] - 19s 5ms/step


[0.016396013841768378, 0.9960425426663368]

In [0]:
ROTPRODataTest=DatasetHelper('oranges','Test')
ROTPRODataTest.loadDataset()

[09/22 07:50:44]  STREAM b'IHDR' 16 13
[09/22 07:50:44]  STREAM b'IDAT' 41 8192
[09/22 07:50:44]  STREAM b'IHDR' 16 13
[09/22 07:50:44]  STREAM b'IDAT' 41 8192
[09/22 07:50:45]  STREAM b'IHDR' 16 13
[09/22 07:50:45]  STREAM b'IDAT' 41 8192
[09/22 07:50:45]  STREAM b'IHDR' 16 13
[09/22 07:50:45]  STREAM b'IDAT' 41 8192
[09/22 07:50:46]  STREAM b'IHDR' 16 13
[09/22 07:50:46]  STREAM b'IDAT' 41 8192
[09/22 07:50:46]  STREAM b'IHDR' 16 13
[09/22 07:50:46]  STREAM b'IDAT' 41 8192
[09/22 07:50:46]  STREAM b'IHDR' 16 13
[09/22 07:50:46]  STREAM b'IDAT' 41 8192
[09/22 07:50:46]  STREAM b'IHDR' 16 13
[09/22 07:50:46]  STREAM b'IDAT' 41 8192
[09/22 07:50:47]  STREAM b'IHDR' 16 13
[09/22 07:50:47]  STREAM b'IDAT' 41 8192
[09/22 07:50:47]  STREAM b'IHDR' 16 13
[09/22 07:50:47]  STREAM b'IDAT' 41 8192
[09/22 07:50:47]  STREAM b'IHDR' 16 13
[09/22 07:50:47]  STREAM b'IDAT' 41 8192
[09/22 07:50:48]  STREAM b'IHDR' 16 13
[09/22 07:50:48]  STREAM b'IDAT' 41 8192
[09/22 07:50:48]  STREAM b'IHDR' 16 13
[

In [0]:
labelsReshapedTest=ROTPRODataTest.labels.reshape(ROTPRODataTest.labels.shape[0],1)

In [0]:
model.evaluate(ROTPRODataTest.input,labelsReshapedTest)

791/791 [==============================] - 4s 5ms/step


[0.1540470786734723, 0.9709228824273072]

In [0]:
# 

# Create APIS


In [0]:
def detect_spoiled(img):
  